In [3]:
import json
import sys
import os
sys.path.insert(0, os.path.abspath('..'))
from collections import deque, defaultdict
from Simulators.cube_model_positions_only import Cube
import copy
import numpy as np

In [7]:
def generate_move_table():
    """
    Generates a table mapping each move to the positions it affects and their new positions.

    Returns:
        dict: A dictionary where keys are move names and values are dictionaries
              mapping initial positions to final positions after the move.
    """
    cube = Cube()  # Create a fresh cube in solved_cube state
    movement_table = {}

    # Positions that are not the center and thus their pieces' positions are tracked
    tracked_positions = [(i,j,k) for i in range(0,3) for j in range(0,3) for k in range(0,3)]

    # For each possible move
    for move in cube.move_map.keys():
        # Get fresh cube for each move calculation
        test_cube = Cube()

        # Record initial piece IDs at each tracked position
        initial_piece_ids = {}
        for pos in tracked_positions:
            initial_piece_ids[pos] = test_cube._get_piece_at_position(pos)

        # Apply the move
        test_cube.apply_moves(move)

        # Record final positions based on moved piece IDs
        movements = {}
        for initial_pos in tracked_positions:
            piece_id_to_track = initial_piece_ids[initial_pos]
            final_pos = test_cube._get_position_of_piece(piece_id_to_track) # Find where that piece ended up
            if initial_pos != final_pos:
                movements[initial_pos] = final_pos

        # Store in table
        movement_table[move] = movements

    # Save to file
    with open('../Precomputed_Tables/position_movement_table.json', 'w') as f:
        # Convert tuple positions to strings for JSON serialization
        serializable_table = {}
        for move, position_movements in movement_table.items():
            serializable_movements = {}
            for from_pos, to_pos in position_movements.items():
                from_pos_str = ','.join(map(str, from_pos))
                to_pos_str = ','.join(map(str, to_pos))
                serializable_movements[from_pos_str] = to_pos_str
            serializable_table[move] = serializable_movements

        json.dump(serializable_table, f, indent=2)

In [8]:
generate_move_table()

In [ ]:
def calculate_distance_table(piece_type, filename:str):
    """
    Builds a position graph for the piece type, then BFSes in that position graph
    from the start position to the target position. This ignores the rest of the puzzle.
    The distance table is keyed by positions, not piece IDs.
    """
    solved_cube = Cube()
    all_moves = list(solved_cube.move_map.keys())
    core_moves = copy.deepcopy(all_moves)
    for move in ['M', 'E', 'S', 'm', 'e', 's']:
        core_moves.remove(move)
    valid_positions = solved_cube.edge_positions if piece_type == "edge" else solved_cube.corner_positions

    #load the position_movement_table created in the previous cell:
    try:
        filename_1 = "../Precomputed_Tables/position_movement_table.json"
        with open(filename_1, 'r') as f:
            serializable_table:dict = json.load(f)
    except Exception as e:
        print(f"Failed to load {filename_1}: {e}")

    # serialize the loaded position_movement json table with the signature -
    # {position: {move: new_position}}
    movement_table = defaultdict(dict)
    for move, position_movements in serializable_table.items():
        for from_pos_str, to_pos_str in position_movements.items():
            from_pos = tuple(eval(from_pos_str))
            to_pos = tuple(eval(to_pos_str))
            movement_table[from_pos][move] = to_pos

    distance_table_1 = {}
    for start_pos in valid_positions: # Iterate over positions, not piece_ids
        for target_pos in valid_positions:
            if start_pos == target_pos:
                distance_table_1[(start_pos, target_pos)] = 0
                continue

            if (target_pos, start_pos) in distance_table_1: #symmetry optimization
                distance_table_1[(start_pos, target_pos)] = distance_table_1[(target_pos, start_pos)]
                continue

            # BFS in the position graph from start_pos to target_pos
            visited = set([start_pos])
            queue = deque([(start_pos, 0)])
            found_distance = -1

            while queue:
                current_pos, dist = queue.popleft()
                if current_pos == target_pos:
                    found_distance = dist
                    break
                # Explore all possible moves from current_pos in the graph
                for move in movement_table[current_pos]:
                    if move not in core_moves:
                        continue
                    else:
                        next_pos = movement_table[current_pos][move]
                        if next_pos not in visited:
                            visited.add(next_pos)
                            queue.append((next_pos, dist + 1))

            distance_table_1[(start_pos, target_pos)] = found_distance

    serializable_table = {}
    for pos_pair, dist in distance_table_1.items(): # Key is now position pair
        serializable_table[str(pos_pair)] = dist
    with open(filename, 'w') as f:
        json.dump(serializable_table, f, indent=2)

In [39]:
calculate_distance_table("edge", "../Precomputed_Tables/edge_position_distance_table.json")
calculate_distance_table("corner", "../Precomputed_Tables/corner_position_distance_table.json")

In [ ]:
def calculate_minimum_paths(filename:str, piece_type:str):
    """
    Builds a minimum-path dictionary from every piece of the given piece_type to every other piece of that type and stores it in the given json file.
    Args:
        filename: The name of the json file for the table to be stored in
        piece_type: `'edge'` or `'corner'`
    """
    cube = Cube()
    # Retrieve the serializable distance table of the corresponding piece type:
    if piece_type == 'edge':
        filename_1 = "../Precomputed_Tables/edge_position_distance_table.json"
    else:
        filename_1 = "../Precomputed_Tables/corner_position_distance_table.json"
    try:
        with open(filename_1, "r") as f:
            serializable_table:dict = json.load(f)
    except Exception as e:
        print(f"Error loading file {filename_1}: {e}")

    # Serialize the loaded distance table with the three signatures - 
    # {init_position(tuple): {distance(int): [final_position(tuple), ...]}}
    # {distance(int): {initial_position(tuple): [final_position(tuple), ...]}}
    # {init_position(tuple): {final_position(tuple): distance(int)}}
    distance_table_1 = defaultdict(lambda: defaultdict(list))
    distance_table_2 = defaultdict(lambda: defaultdict(list))
    distance_table_3 = defaultdict(dict)
    for pos_pair, distance in serializable_table.items():
        pos_pair_tuple = tuple(eval(pos_pair))
        init_pos = pos_pair_tuple[0]
        final_pos = pos_pair_tuple[1]
        distance_table_1[init_pos][distance].append(final_pos)
        distance_table_2[distance][init_pos].append(final_pos)
        distance_table_3[init_pos][final_pos] = distance

    #load the position_movement_table created in a previous cell:
    try:
        filename_2 = "../Precomputed_Tables/position_movement_table.json"
        with open(filename_2, 'r') as f:
            serializable_table:dict = json.load(f)
    except Exception as e:
        print(f"Failed to load {filename_2}: {e}")

    # serialize the loaded movement json table with the signature -
    # {position: {new_position: [moves]}}
    movement_table = defaultdict(lambda: defaultdict(list))
    for move, position_movements in serializable_table.items():
        for from_pos_str, to_pos_str in position_movements.items():
            from_pos = tuple(eval(from_pos_str))
            to_pos = tuple(eval(to_pos_str))
            movement_table[from_pos][to_pos].append(move)
    
    # Create the data structure to store the final_output (min_paths) with the signature - 
    # {path_length(int): {position(tuple): {new_position(tuple): [[move_sequence(str)], ...]}}}
    min_paths = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

    # Get possible distances list:
    possible_distances = []
    for position, distances in distance_table_1.items():
            possible_distances.extend(distances.keys())
    possible_distances = sorted(list(set(possible_distances)))
    possible_distances.pop(0)
    
 
    # Begin building paths:

    # build length-one paths:
    for a, b in distance_table_1.items():
        for c in b[1]:
            min_paths[1][a][c] = movement_table[a][c]

    for current_distance in possible_distances:
        for init_pos, positions_for_dists in distance_table_1.items():
            final_positions = positions_for_dists[current_distance]
            if (current_distance-1) in min_paths.keys():
                for previous_level_final_pos, previous_level_paths in min_paths[current_distance-1][init_pos].items():
                    for final_position in final_positions:
                        if distance_table_3[previous_level_final_pos][final_position] == 1:
                            for previous_level_path in previous_level_paths:
                                for move in movement_table[previous_level_final_pos][final_position]:
                                    if move not in previous_level_path:
                                        min_paths[current_distance][init_pos][final_position].append(previous_level_path+move)
    print(min_paths)
    
    
    serializable_table = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
    for distance, x in min_paths.items():
        for init_position, y in x.items():
            for final_position, paths in y.items():
                json_paths = ", ".join([path for path in paths])
                serializable_table[str(distance)][str(init_position)][str(final_position)] = json_paths
    with open(filename, 'w') as f:
        json.dump(serializable_table, f, indent=2)

In [206]:
calculate_minimum_paths("../Precomputed_Tables/edge_min_path_table.json", "edge")
calculate_minimum_paths("../Precomputed_Tables/corner_min_path_table.json", "corner")

defaultdict(<function calculate_minimum_paths.<locals>.<lambda> at 0x0000018B73EA3600>, {1: defaultdict(<function calculate_minimum_paths.<locals>.<lambda>.<locals>.<lambda> at 0x0000018B73EA34C0>, {(0, 0, 1): defaultdict(<class 'list'>, {(0, 1, 0): ['f'], (0, 1, 2): ['F'], (1, 0, 0): ['U'], (1, 0, 2): ['u']}), (0, 1, 0): defaultdict(<class 'list'>, {(0, 0, 1): ['F'], (0, 2, 1): ['f'], (1, 0, 0): ['l'], (1, 2, 0): ['L']}), (0, 1, 2): defaultdict(<class 'list'>, {(0, 0, 1): ['f'], (0, 2, 1): ['F'], (1, 0, 2): ['R'], (1, 2, 2): ['r']}), (0, 2, 1): defaultdict(<class 'list'>, {(0, 1, 0): ['F'], (0, 1, 2): ['f'], (1, 2, 0): ['d'], (1, 2, 2): ['D']}), (1, 0, 0): defaultdict(<class 'list'>, {(0, 0, 1): ['u'], (0, 1, 0): ['L'], (2, 0, 1): ['U'], (2, 1, 0): ['l']}), (1, 0, 2): defaultdict(<class 'list'>, {(0, 0, 1): ['U'], (0, 1, 2): ['r'], (2, 0, 1): ['u'], (2, 1, 2): ['R']}), (1, 2, 0): defaultdict(<class 'list'>, {(0, 1, 0): ['l'], (0, 2, 1): ['D'], (2, 1, 0): ['L'], (2, 2, 1): ['d']}), (1,